В этом примере мы покажем, как с помощью API Яндекс.Геокодера можно преобразовать адреса в геокоординаты, с которым можно работать в [Datalens](https://datalens.yandex.ru)

## Установка библиотек

 * requests для API Геокодера
 * clickhouse-driver для Clickhouse

In [1]:
%pip install requests
%pip install pandas

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install clickhouse-driver

Note: you may need to restart the kernel to use updated packages.


## Получение ключа для API Геокодера

В переменную API_KEY нужно записать свой ключ для API Геокодера.
Получить ключ можно по [ссылке](https://yandex.ru/dev/maps/geocoder/)

In [3]:

API_KEY = '5a59654e-14a4-4b47-8c97-48976539273f'


## Работа с API Геокодера

Напишем небольшой класс для работы с Геокодером

In [4]:
import requests
from dataclasses import dataclass

@dataclass
class YandexGeocoder:
    api_key: str
    geocoder_url: str = 'https://geocode-maps.yandex.ru/1.x'

    def adress_to_geopoint(self, address: str) -> str:
        """
        Преобразование адреса в геокоординаты в формате Datalens
        """
        response = requests.get(self.geocoder_url, params={
            'apikey': self.api_key,
            'geocode': address,
            'format': 'json',
        })
        response.raise_for_status()

        result = response.json()['response']['GeoObjectCollection']['featureMember']
        if not result:
            return None

        lat, lon = result[0]['GeoObject']['Point']['pos'].split(' ')
        return self._to_datalens_format(lon, lat)
    
    def _to_datalens_format(self, lon, lat):
        return f'[{lon},{lat}]'

## Получение данных

Будем работать с данными из демонстрационного Clickhouse [Datalens](https://datalens.yandex.ru).

На первом шаге подготовим клиент Clickhouse

In [5]:
from clickhouse_driver import Client as CHClient

ch_client = CHClient(
    'rc1a-ckg8nrosr2lim5iz.mdb.yandexcloud.net',
    user='samples_ro',
    password='MsgfcjEhJk',
    database='samples',
    port=9440,
    secure=True,
)

Затем выгрузим данные из таблицы в переменную ch_data

In [6]:
import pandas as pd
all_files = ["out.csv"]
df = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
used = set()
ch_data = []
for row in df['city']:
    if isinstance(row, str) and row not in used:
        ch_data.append(row)
        used.add(row)
    
ch_data    

['Москва',
 'Ростов',
 'Ярославль',
 'Ярославль (деревня)',
 'Ливны',
 'Казань',
 'Рыбинск',
 'Tokyo',
 'Брейтово',
 'Санкт-Петербург',
 'Нижний Новгород',
 'Уфа',
 'Саратов',
 'Омск',
 'Ступино',
 'Кисловодск']

In [7]:
 
for i in range(len(ch_data)):
    ch_data[i] = "Russia, " + ch_data[i]


In [8]:
ch_data

['Russia, Москва',
 'Russia, Ростов',
 'Russia, Ярославль',
 'Russia, Ярославль (деревня)',
 'Russia, Ливны',
 'Russia, Казань',
 'Russia, Рыбинск',
 'Russia, Tokyo',
 'Russia, Брейтово',
 'Russia, Санкт-Петербург',
 'Russia, Нижний Новгород',
 'Russia, Уфа',
 'Russia, Саратов',
 'Russia, Омск',
 'Russia, Ступино',
 'Russia, Кисловодск']

## Геокодирование

Преобразуем адреса магазинов из колонки ShopAddress в геокоординаты

In [9]:
%pip install yandex-geocoder
from yandex_geocoder import Client

geocoder = Client(api_key=API_KEY)
ch_data

Note: you may need to restart the kernel to use updated packages.


['Russia, Москва',
 'Russia, Ростов',
 'Russia, Ярославль',
 'Russia, Ярославль (деревня)',
 'Russia, Ливны',
 'Russia, Казань',
 'Russia, Рыбинск',
 'Russia, Tokyo',
 'Russia, Брейтово',
 'Russia, Санкт-Петербург',
 'Russia, Нижний Новгород',
 'Russia, Уфа',
 'Russia, Саратов',
 'Russia, Омск',
 'Russia, Ступино',
 'Russia, Кисловодск']

In [10]:
encoded_data = []
for adress in ch_data:
    s = geocoder.coordinates(adress)
    s = list(map(float, s[::-1]))
    encoded_data.append(s)
print(encoded_data)

[[55.755819, 37.617644], [57.18587, 39.414526], [57.626559, 39.893813], [57.626559, 39.893813], [52.426549, 37.60805], [55.796127, 49.106414], [58.04864, 38.855711], [55.719815, 37.694064], [58.302689, 37.866666], [59.938676, 30.314494], [56.326797, 44.006516], [54.735152, 55.958736], [51.533562, 46.034266], [54.989347, 73.368221], [54.88628, 38.078237], [43.905518, 42.715718]]


In [11]:
len(encoded_data)

16

In [12]:
import pandas as pd

In [13]:
ch_data2 = [w[8:] for w in ch_data]
encoded_data

[[55.755819, 37.617644],
 [57.18587, 39.414526],
 [57.626559, 39.893813],
 [57.626559, 39.893813],
 [52.426549, 37.60805],
 [55.796127, 49.106414],
 [58.04864, 38.855711],
 [55.719815, 37.694064],
 [58.302689, 37.866666],
 [59.938676, 30.314494],
 [56.326797, 44.006516],
 [54.735152, 55.958736],
 [51.533562, 46.034266],
 [54.989347, 73.368221],
 [54.88628, 38.078237],
 [43.905518, 42.715718]]

In [14]:
data = [[ch_data2[i], encoded_data[i]] for i in range(len(encoded_data))]
df = pd.DataFrame(data, columns=['город', 'координаты'])
df

,город,координаты
0,Москва,"[55.755819, 37.617644]"
1,Ростов,"[57.18587, 39.414526]"
2,Ярославль,"[57.626559, 39.893813]"
3,Ярославль (деревня),"[57.626559, 39.893813]"
4,Ливны,"[52.426549, 37.60805]"
5,Казань,"[55.796127, 49.106414]"
6,Рыбинск,"[58.04864, 38.855711]"
7,Tokyo,"[55.719815, 37.694064]"
8,Брейтово,"[58.302689, 37.866666]"
9,Санкт-Петербург,"[59.938676, 30.314494]"


In [15]:
df.to_csv("coord_city5.csv")